# Assignment 1

This Jupyter Notebook document is our implementation of Assignment 1. 


In [1]:
# !pip install pandas 
# !pip install numpy
# !pip install matplotlib

In [2]:
# !pip install wget

# Importing Libraries

    

In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import zipfile
import os

# Task 1: Data Loading and Splitting
* **Download** the corpus.
* **Encode** the corpus into a pandas.DataFrame object.
* **Split** it in training, validation, and test sets.

[Penn TreeBank corpus](https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip).

In [4]:
# Download the data from Penn TreeBank corpus 

address = 'https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip'
urllib.request.urlretrieve(address, 'dependency_treebank.zip')
# Unzip the data
with zipfile.ZipFile('dependency_treebank.zip', 'r') as zip_ref:
    zip_ref.extractall()


In [5]:
# Encode the corpus in a dataframe object
list_train = [] # documents 1-100
list_test = [] # documents 101-150
list_val = [] # documents 151-199
for filename in os.listdir('dependency_treebank/'):
    if filename.endswith('.dp'):
        with open('dependency_treebank/' + filename, 'r') as f:
            lines = f.readlines()
            for line in lines:
                if int(filename[4:8]) <= 100:
                    list_train.append(line.split()[:-1])
                elif int(filename[4:8]) <= 150:
                    list_test.append(line.split()[:-1])
                elif int(filename[4:8]) <= 199:
                    list_val.append(line.split()[:-1])
         
print(len(list_train))
print(len(list_test))
print(len(list_val))

print(list_train[0:20])

# create a dataframe object
df_train = pd.DataFrame(list_train)
df_test = pd.DataFrame(list_test)
df_val = pd.DataFrame(list_val)

print(df_train.shape)
print(df_test.shape)
print(df_val.shape)

print(df_train.head(5))
print(df_test.head(5))
print(df_val.head(5))

print(df_train.values[:,0])

49219
32432
16148
[['Pierre', 'NNP'], ['Vinken', 'NNP'], [',', ','], ['61', 'CD'], ['years', 'NNS'], ['old', 'JJ'], [',', ','], ['will', 'MD'], ['join', 'VB'], ['the', 'DT'], ['board', 'NN'], ['as', 'IN'], ['a', 'DT'], ['nonexecutive', 'JJ'], ['director', 'NN'], ['Nov.', 'NNP'], ['29', 'CD'], ['.', '.'], [], ['Mr.', 'NNP']]
(49219, 2)
(32432, 2)
(16148, 2)
        0    1
0  Pierre  NNP
1  Vinken  NNP
2       ,    ,
3      61   CD
4   years  NNS
              0    1
0             A   DT
1  House-Senate  NNP
2    conference   NN
3      approved  VBD
4         major   JJ
           0    1
0  Intelogic  NNP
1      Trace  NNP
2       Inc.  NNP
3          ,    ,
4        San  NNP
['Pierre' 'Vinken' ',' ... 'has' 'faced' '.']


# Task 2: Text encoding

To train a neural POS tagger, you first need to encode text into numerical format.

### Instructions

* Embed words using **GloVe embeddings**.
* You are **free** to pick any embedding dimension.
* [Optional] You are free to experiment with text pre-processing: **make sure you do not delete any token!**

In [6]:
# !pip install gensim

In [7]:
import gensim
import gensim.downloader as gloader

def load_embedding_model(embedding_dimension: int = 50) -> gensim.models.keyedvectors.KeyedVectors:
    download_path = ""
    download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)
        
    try:
        emb_model = gloader.load(download_path)
    except ValueError as e:
        print("Invalid embedding model name! Check the embedding dimension:")
        print("Glove: 50, 100, 200, 300")
        raise e

    return emb_model

c:\Users\39328\anaconda3\lib\site-packages\scipy\__init__.py:173: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
# Glove -> 50, 100, 200, 300
embedding_model = load_embedding_model(embedding_dimension=50)

# Task 3: Model definition

You are now tasked to define your neural POS tagger.

### Instructions

* **Baseline**: implement a Bidirectional LSTM with a Dense layer on top.
* You are **free** to experiment with hyper-parameters to define the baseline model.

* **Model 1**: add an additional LSTM layer to the Baseline model.
* **Model 2**: add an additional Dense layer to the Baseline model.

* **Do not mix Model 1 and Model 2**. Each model has its own instructions.

**Note**: if a document contains many tokens, you are **free** to split them into chunks or sentences to define your mini-batches.

In [2]:
!pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/fa/47/1a7daf04f40715fc1cdc6f1cc3200228a556d06c843e6ceb58883b745e1b/torch-2.1.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/81/54/84d42a0bee35edba99dee7b59a8d4970eccdd44b99fe728ed912106fc781/filelock-3.13.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions from https://files.pythonhosted.org/packages/24/21/7d397a4b7934ff4028987914ac1044d3b7d52712f30e2ac7a2ae5bc86dd0/typing_extensions-4.8.0-py3-none-any.whl.metadata
     ---------------------------------------- 5.7/5.7 MB 14.7 MB/s eta 0:00:00
  Obtaining dependency information for fsspec from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
     ------------------------------------- 536.2/536.2 kB 32.9 MB/s eta 0:00:00
   ------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\alepa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install --upgrade torch torchvision
!pip install --upgrade typing-extensions

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/59/af/426c2b90f5c4f8aba778746465af9e662680570e950e02379e91c6138285/torchvision-0.16.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/a2/a0/4af29e22cb5942488cf45630cbdd7cefd908768e69bdd90280842e4e8529/charset_normalizer-3.3.2-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/d2/b2/b157855192a68541a91ba7b2bbcb91f1b4faa51f8bae38d8005c034be524/urllib3-2.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/4c/dd/2234eab22353ffc7d94e8d1317


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\alepa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\alepa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Baseline





In [4]:
import torch
import torch.nn as nn

class Baseline(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(Baseline, self).__init__()
        
        # Bidirectional LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        
        # Dense layer for classification
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # Multiply by 2 for bidirectional
        
    def forward(self, x):
        # Forward pass through Bidirectional LSTM layer
        out, _ = self.lstm(x)
        
        # Get the output from the last time step (if you want to use the output from all time steps, modify this accordingly)
        out = out[:, -1, :]
        
        # Forward pass through Dense layer for classification
        out = self.fc(out)
        return out

In [ ]:
# Create an instance of the BidirectionalLSTM model
model = Baseline(input_size, 64, 1, num_classes)

# Print the model architecture
print(model)

### Model 1

In [5]:
class Model1(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(Model1, self).__init__()
        
        # Bidirectional LSTM layers
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(hidden_size * 2, hidden_size, num_layers, batch_first=True, bidirectional=True)
        
        # Dense layer for classification
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # Multiply by 2 for bidirectional
        
    def forward(self, x):
        # Forward pass through first Bidirectional LSTM layer
        out, _ = self.lstm1(x)
        
        # Forward pass through second Bidirectional LSTM layer
        out, _ = self.lstm2(out)
        
        # Get the output from the last time step (if you want to use the output from all time steps, modify this accordingly)
        out = out[:, -1, :]
        
        # Forward pass through Dense layer for classification
        out = self.fc(out)
        return out


In [ ]:
# Create an instance of the BidirectionalLSTM model
model = Model1(input_size, 64, 1, num_classes)

# Print the model architecture
print(model)

### Model 2

In [6]:
class Model2(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(Model2, self).__init__()
        
        # Bidirectional LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        
        # First Dense layer
        self.fc1 = nn.Linear(hidden_size * 2, hidden_size)  # Multiply by 2 for bidirectional
        
        # Second Dense layer for classification
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Forward pass through Bidirectional LSTM layer
        out, _ = self.lstm(x)
        
        # Get the output from the last time step (if you want to use the output from all time steps, modify this accordingly)
        out = out[:, -1, :]
        
        # Forward pass through the first Dense layer
        out = self.fc1(out)
        
        # ReLU activation function (you can use other activation functions as well)
        out = nn.functional.relu(out)
        
        # Forward pass through the second Dense layer for classification
        out = self.fc2(out)
        return out

In [ ]:
# Create an instance of the BidirectionalLSTM model
model = Model2(input_size, 64, 1, num_classes)

# Print the model architecture
print(model)

# Task 4: Metrics

Before training the models, you are tasked to define the evaluation metrics for comparison.

### Instructions

* Evaluate your models using macro F1-score, compute over **all** tokens.
* **Concatenate** all tokens in a data split to compute the F1-score. (**Hint**: accumulate FP, TP, FN, TN iteratively) 
* **Do not consider punctuation and symbol classes** $\rightarrow$ [What is punctuation?](https://en.wikipedia.org/wiki/English_punctuation)

**Note**: What about OOV tokens?
   * All the tokens in the **training** set that are not in GloVe are **not** considered as OOV
   * For the remaining tokens (i.e., OOV in the validation and test sets), you have to assign them a **static** embedding.
   * You are **free** to define the static embedding using any strategy (e.g., random, neighbourhood, 

In [12]:
from typing import List, Dict

def check_OOV_terms(embedding_model: gensim.models.keyedvectors.KeyedVectors, word_listing: List[str]):
    
    embedding_vocabulary = set(embedding_model.key_to_index.keys())
    oov = set(word_listing).difference(embedding_vocabulary)
    return list(oov)

In [13]:
oov_terms = check_OOV_terms(embedding_model, df_train.values[:,0])
oov_percentage = float(len(oov_terms)) * 100 / len(df_train.values[:,0])

print(oov_percentage)

4.768483715638269


In [14]:
def build_embedding_matrix(embedding_model: gensim.models.keyedvectors.KeyedVectors,
                           embedding_dimension: int,
                           word_to_idx: Dict[str, int],
                           vocab_size: int,
                           oov_terms: List[str]) -> np.ndarray:
    
    embedding_matrix = np.zeros((vocab_size, embedding_dimension), dtype=np.float32)
    for word, idx in tqdm(word_to_idx.items()):
        try:
            embedding_vector = embedding_model[word]
        except (KeyError, TypeError):
            embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

        embedding_matrix[idx] = embedding_vector

    return embedding_matrix

In [15]:
embedding_dimension = 50
embedding_matrix = build_embedding_matrix(embedding_model, embedding_dimension, word_to_idx, len(word_to_idx), oov_terms)
print(f"Embedding matrix shape: {embedding_matrix.shape}")

NameError: name 'word_to_idx' is not defined

# Task 5: Training and Evaluation

You are now tasked to train and evaluate the Baseline, Model 1, and Model 2.

### Instructions

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Compute metrics on the validation set.
* Pick **at least** three seeds for robust estimation.
* Pick the **best** performing model according to the observed validation set performance.